In [20]:
import datetime as dt
import time
import logging as log
from multiprocessing import Pool, TimeoutError

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb
%run AssetPainter.ipynb

class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def build_operation_file_path(self,symbol,regression_end_date):
        return report_dir+'operations_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def build_deal_file_path(self,symbol,regression_end_date):
        return report_dir+'deals_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
        
    def build_asset_file_path(self,symbol,regression_end_date):
        return report_dir+'account_asset_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def persist(self,account,operation_df,symbol,regression_end_date):
        account.get_asset_df().to_csv(self.build_asset_file_path(symbol,regression_end_date),index=False)
        account.deal_df.to_csv(self.build_deal_file_path(symbol,regression_end_date),index=False)
        operation_df.to_csv(self.build_operation_file_path(symbol,regression_end_date), index=False)
        
    def draw_asset(self,account,operation_df,symbol,regression_end_date):
        if(self.policy_loader.enable_painter()==True):
            painter = AssetPainter()

            asset_file = self.build_asset_file_path(symbol,regression_end_date)
            painter.draw_month_profit_bars(asset_file)
            painter.draw_audit_asset_bars(asset_file)

            operation_file=self.build_operation_file_path(symbol,regression_end_date)
            painter.draw_operation_asset_bars(operation_file)
    
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        start = time.time()
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\n'+row['symbol']+' section '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                
        operation_df = operation.get_operations()
        self.persist(account,operation_df,symbol,regression_end_date)
        self.draw_asset(account,operation_df,symbol,regression_end_date)
        
        evaluation = Evaluation(account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                         operation_df)

        end = time.time()
        if(self.debug==False):
            print(symbol+' regression cost time ' + str(round(end-start,1))+' seconds')
        return evaluation_df
    
    
    def start_sync_regression(self,scale,regression_end_date):
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        for symbol in symbols:
            one_evaluation = self.start_regression_on_one_stock(symbol,regression_end_date)
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
           
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    
    def start_regression(self,scale,regression_end_date,process_num):
        print('context:' + str(self.context))
        
        if(process_num==1):
            return self.start_sync_regression(scale,regression_end_date)
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        pool = Pool(processes=process_num)
        tasks = []
        for symbol in symbols:
            task = pool.apply_async(self.start_regression_on_one_stock, (symbol,regression_end_date))
            tasks.append(task)
            
        for task in tasks:
            try:
                one_evaluation = task.get()
                evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            except Exception as e:
                print(str(e))
        
        pool.close()
        pool.join()
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    

In [21]:
regression = RocketRegression('rocket_break_v5_enable_stock_level_control.json')

regression_end='2019-05-30'
scale=300
process_num=24

evaluation_df = regression.start_regression(scale,regression_end,process_num)
evaluation_df.describe()

context:{'short': 30, 'mid': 240, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002506 regression cost time 5.2 seconds
SZ#002346 regression cost time 16.0 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002504 regression cost time 22.5 seconds
SZ#002345 regression cost time 23.0 seconds
SZ#002343 regression cost time 24.3 seconds
SZ#002505 regression cost time 24.6 seconds
SZ#002181 regression cost time 29.5 seconds
SZ#002183 regression cost time 32.3 seconds
SZ#002182 regression cost time 35.8 seconds
SZ#002184 regression cost time 35.6 seconds
SZ#000809 regression cost time 39.9 seconds
SZ#000582 regression cost time 43.7 seconds
SZ#000803 regression cost time 45.0 seconds


...

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:132: RuntimeWarning: divide by zero encountered in double_scalars


SZ#002381 regression cost time 20.8 seconds
SZ#002056 regression cost time 55.7 seconds
SZ#002542 regression cost time 36.5 seconds
SZ#000043 regression cost time 62.8 seconds
SZ#000623 regression cost time 64.2 seconds
SZ#000861 regression cost time 64.1 seconds
SZ#000039 regression cost time 96.7 seconds
attempt to get argmax of an empty sequence
SZ#002059 regression cost time 22.6 seconds
SZ#002543 regression cost time 28.7 seconds
SZ#002382 regression cost time 22.0 seconds
SZ#000045 regression cost time 60.0 seconds
SZ#002058 regression cost time 45.9 seconds
SZ#002220 regression cost time 29.6 seconds
SZ#002383 regression cost time 18.8 seconds
SZ#002545 regression cost time 17.3 seconds
SZ#000626 regression cost time 54.2 seconds
SZ#002219 regression cost time 50.5 seconds


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in double_scalars


SZ#000860 regression cost time 92.7 seconds
attempt to get argmax of an empty sequence
SZ#002544 regression cost time 38.6 seconds
SZ#002546 regression cost time 20.2 seconds
SZ#000627 regression cost time 53.6 seconds
SZ#000628 regression cost time 43.9 seconds
SZ#002060 regression cost time 42.0 seconds
SZ#002385 regression cost time 17.5 seconds
SZ#002221 regression cost time 41.1 seconds
SZ#000868 regression cost time 54.5 seconds
SZ#000046 regression cost time 76.8 seconds
SZ#002547 regression cost time 18.3 seconds
SZ#002384 regression cost time 30.3 seconds
SZ#002222 regression cost time 31.0 seconds
SZ#000049 regression cost time 51.5 seconds
SZ#002062 regression cost time 26.2 seconds
SZ#002061 regression cost time 39.5 seconds
SZ#000048 regression cost time 63.5 seconds
attempt to get argmax of an empty sequence
SZ#000875 regression cost time 41.1 seconds
SZ#002223 regression cost time 28.1 seconds
SZ#000629 regression cost time 44.2 seconds
SZ#000055 regression cost time 39.

,in_hand_asset_drawdown,loss_times,float_profit_drawdown,avg_year_return,year_regular_return,mean_year_return,std_year_return,mean_month_return,std_month_return,profit_month_rate,win_rate,mar,sharp,rar,deal_count
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,281.000000,281.000000,288.000000
mean,15.102049,4.361111,29.457049,13.786181,68.348090,4.456895,15.309558,1.568885,9.730934,56.206146,50.081632,21.690243,-inf,98.805693,17.072917
std,9.931492,2.390781,9.257750,27.130462,134.996065,6.722345,7.174066,2.365631,6.673180,10.011097,15.135264,100.478075,NaN,709.294981,7.357347
min,0.100000,0.000000,12.860000,-8.000000,-52.110000,-10.207500,1.569777,-2.500000,2.122522,18.180000,14.290000,-0.500000,-inf,-5965.714286,2.000000
25%,8.750000,3.000000,23.717500,0.630000,3.985000,0.051250,9.911204,0.080829,6.101134,50.000000,39.070000,0.050000,-1.122082,-55.333333,11.000000
50%,13.465000,4.000000,28.070000,6.395000,29.830000,3.480000,14.329680,0.990171,8.326808,56.560000,50.000000,0.465000,0.114761,52.142857,16.000000
75%,20.925000,6.000000,33.890000,16.577500,80.035000,8.105000,20.339503,2.211071,10.803586,62.215000,60.000000,1.922500,0.908754,214.285714,22.000000
max,50.700000,12.000000,82.440000,266.870000,1423.500000,34.013333,37.076877,13.929697,55.862745,83.330000,100.000000,908.230000,8.594067,5830.000000,38.000000


In [ ]:
print('Helo')

In [22]:
evaluation_df[:10]

,symbol,start_date,end_date,policy_name,short,mid,in_hand_asset_drawdown,avg_drawdown,loss_times,float_profit_drawdown,...,mean_year_return,std_year_return,mean_month_return,std_month_return,profit_month_rate,win_rate,mar,sharp,rar,deal_count
0,SZ#002547,2011-02-18,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,3.04,3.0434,2,21.64,...,2.625000,7.444580,0.838636,5.911285,72.73,75.00,1.01,-6.543303,-45.000000,8
1,SZ#002385,2010-04-09,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,17.80,17.8018,4,20.92,...,-4.857500,8.208440,-1.142143,7.319472,28.57,50.00,-0.20,-3.488050,90.000000,10
2,SZ#002223,2008-04-18,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,23.23,23.2343,6,27.99,...,-0.825000,4.228426,0.025102,3.539487,71.43,23.08,-0.03,-2.343054,-20.000000,13
3,SZ#002062,2006-08-16,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,23.67,19.9063,10,24.51,...,-0.534000,13.834413,-0.057931,8.120867,55.17,29.41,0.02,-3.753244,99.000000,17
4,SZ#000630,2000-02-14,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,11.73,6.05353,4,24.22,...,3.274000,21.209153,1.266154,10.357106,41.03,56.52,1.28,0.961820,208.000000,23
5,SZ#000055,2000-02-14,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,22.65,20.8198,5,41.77,...,-3.877500,9.477157,-0.977143,6.516553,50.00,50.00,-0.10,-1.174817,181.428571,24
6,SZ#002546,2011-02-11,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,6.97,6.97495,5,24.02,...,1.013333,9.121104,0.081250,7.279469,50.00,53.85,0.25,-0.766032,960.000000,13
7,SZ#002384,2010-04-09,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,8.71,4.51999,4,28.76,...,4.668000,11.817507,0.743793,10.319278,55.17,53.33,0.71,2.317125,-108.000000,15
8,SZ#002222,2008-03-19,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,10.21,9.82451,5,27.96,...,0.673333,9.053131,0.077742,5.189795,54.84,38.89,0.12,-0.867855,25.238095,18
9,SZ#002061,2006-08-16,2019-02-18,rocket_break_v5_enable_stock_level_control.json,30,240,13.06,13.0084,10,29.33,...,3.158571,14.641557,1.260303,9.573058,45.45,36.84,0.46,0.003910,32.857143,19
